In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.api import qqplot, add_constant
import statsmodels.formula.api as smf

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from sklearn.preprocessing import StandardScaler
from statsmodels.tools.eval_measures import rmse
import matplotlib
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import export_graphviz  
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import scale, minmax_scale, robust_scale

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

%matplotlib inline
matplotlib.rc('font', family='AppleGothic')
plt.rc('font', family='AppleGothic')

matplotlib.rc('axes', unicode_minus = False)
from scipy.stats import chi2_contingency
from scipy import stats

from statsmodels.api import Logit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기 및 컬럼 변경

In [ ]:
cook = pd.read_csv("latestversioncook.csv",encoding='euc-kr')

# 불량여부를 불량으로 컬럼명 변환
cook.rename(columns = {'불량여부':'불량'},inplace=True)
cook.rename(columns = {'발생시간파생':'오류시간Hour'},inplace=True)

## 불량여부 1,0 치환

In [ ]:
cook['불량'] = np.where(cook['불량']=='불량',1 ,0)

In [ ]:
cook.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,생산시간,생산일자,불량,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명
0,3,C001,쿠커 #1,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,...,70.37,2021-01-03,0,0,0,0,0.0,0,0,0
1,1,C002,쿠커 #2,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,...,70.83,2021-01-03,0,0,0,0,0.0,0,0,0
2,13,C003,쿠커 #3,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,...,73.55,2021-01-03,0,0,0,0,0.0,0,0,0
3,7,C005,2동 쿠커 #4,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,...,74.81,2021-01-03,0,0,0,0,0.0,0,0,0
4,2,C007,프리머 #1,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179318,114.079318,149.928068,23.899,...,73.40,2021-01-03,0,0,0,0,0.0,0,0,0


In [ ]:
cook['불량'].value_counts()

0    22900
1     1072
Name: 불량, dtype: int64

## 품목별

In [ ]:
product = cook.groupby('품목명')['품목명'].count()
product = pd.DataFrame(product)
product = pd.DataFrame(product,index=list(product['품목명'].index))
product = product.reset_index()

In [ ]:
def match_item(a,b):
    for i in range(len(b)):
        if a == b['index'][i]:
            ans = b['품목명'][i]
            return ans

cook['품목명합계'] = cook.apply(lambda x : match_item(x['품목명'],product),axis=1)

In [ ]:
cook.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,생산일자,불량,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명,품목명합계
0,3,C001,쿠커 #1,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,...,2021-01-03,0,0,0,0,0.0,0,0,0,26
1,1,C002,쿠커 #2,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,...,2021-01-03,0,0,0,0,0.0,0,0,0,22
2,13,C003,쿠커 #3,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,...,2021-01-03,0,0,0,0,0.0,0,0,0,15
3,7,C005,2동 쿠커 #4,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,...,2021-01-03,0,0,0,0,0.0,0,0,0,34
4,2,C007,프리머 #1,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179318,114.079318,149.928068,23.899,...,2021-01-03,0,0,0,0,0.0,0,0,0,61


In [ ]:
cook.to_csv("품목.csv",index=False,encoding='euc-kr')

In [ ]:
cook1 = pd.read_csv("품목.csv",encoding='euc-kr')
cook1.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,생산일자,불량,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명,품목명합계
0,3,C001,쿠커 #1,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,...,2021-01-03,0,0,0,0,0.0,0,0,0,26
1,1,C002,쿠커 #2,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,...,2021-01-03,0,0,0,0,0.0,0,0,0,22
2,13,C003,쿠커 #3,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,...,2021-01-03,0,0,0,0,0.0,0,0,0,15
3,7,C005,2동 쿠커 #4,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,...,2021-01-03,0,0,0,0,0.0,0,0,0,34
4,2,C007,프리머 #1,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179318,114.079318,149.928068,23.899,...,2021-01-03,0,0,0,0,0.0,0,0,0,61


## 생산현황

In [ ]:
x = cook1.drop_duplicates(subset=['품목명']) 
x = x.sort_values('품목명합계',ascending=False)
x.head(50)
x[x['품목명합계']>10].head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,생산일자,불량,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명,품목명합계
17017,12,D088,냉동밥 내포장#1,1BA03389,게살볶음밥E,W005,71.907033,126.207033,162.374933,24.112,...,2018-11-08,0,0,0,0,0.0,0,0,0,362
104,10,D005,3.2마요,16B06003,골드마요네즈3.2kg,W005,71.555133,107.655133,143.270683,23.744,...,2021-01-07,0,0,0,0,0.0,0,0,0,309
8095,13,D088,냉동밥 내포장#1,1BA03051,쉐프솔루션베이스볶음밥(시즌)500g,W005,70.777656,133.777656,168.796506,23.346,...,2017-02-16,0,0,0,0,0.0,0,0,0,252
8091,9,D087,냉동밥 내포장#2,1BA03158,비비고불고기비빔밥200g,W005,71.226196,91.926196,126.899896,23.316,...,2017-02-16,0,0,0,0,0.0,0,0,0,239
8169,11,D087,냉동밥 내포장#2,1BA03200,비비고새우볶음밥210g(알밥),W005,71.551792,92.751792,128.333892,23.721,...,2017-02-22,0,0,0,0,0.0,0,0,0,230


## 생산량 10개 이상 조건

In [ ]:
df = cook1[(cook1['품목명합계'] > 10) & (cook1['불량'] == 1)]
df = df.groupby('품목명')
df = df.size()
df = pd.DataFrame(df)
df = df.reset_index()
df.rename(columns={"품목명": "index", 0: "품목명"},inplace=True)
df

,index,품목명
0,(1)게살볶음밥E,1
1,(20년)아몬드&호두드레싱-N225g,1
2,(20년)참깨&흑임자드레싱-N105g,3
3,(쿠팡)곰곰간장버터새우볶음밥,1
4,(쿠팡)곰곰갈비맛치킨밥,1
...,...,...
307,홈스타일발사믹드레싱,1
308,홈스타일오리엔탈드레싱,2
309,홍익돈까스 야채볶음밥,7
310,홍익돈까스 야채볶음밥(재),4


In [ ]:
def match_scale(a,b):
    for i in range(len(b)):
        if a == b['index'][i]:
            ans = b['품목명'][i]
            return ans

cook1['품목명불량갯수'] = cook1.apply(lambda x : match_scale(x['품목명'],df),axis=1)
cook1['품목명불량갯수'].fillna(0,inplace=True)

In [ ]:
cook1.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,불량,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명,품목명합계,품목명불량갯수
0,3,C001,쿠커 #1,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,...,0,0,0,0,0.0,0,0,0,26,4.0
1,1,C002,쿠커 #2,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,...,0,0,0,0,0.0,0,0,0,22,0.0
2,13,C003,쿠커 #3,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,...,0,0,0,0,0.0,0,0,0,15,1.0
3,7,C005,2동 쿠커 #4,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,...,0,0,0,0,0.0,0,0,0,34,0.0
4,2,C007,프리머 #1,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179318,114.079318,149.928068,23.899,...,0,0,0,0,0.0,0,0,0,61,2.0


## 불량률 구하기

In [ ]:
cook1['품목별불량률'] = round((cook1['품목명불량갯수'] / cook1['품목명합계']),2)
cook1.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명,품목명합계,품목명불량갯수,품목별불량률
0,3,C001,쿠커 #1,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,...,0,0,0,0.0,0,0,0,26,4.0,0.15
1,1,C002,쿠커 #2,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,...,0,0,0,0.0,0,0,0,22,0.0,0.00
2,13,C003,쿠커 #3,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,...,0,0,0,0.0,0,0,0,15,1.0,0.07
3,7,C005,2동 쿠커 #4,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,...,0,0,0,0.0,0,0,0,34,0.0,0.00
4,2,C007,프리머 #1,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179318,114.079318,149.928068,23.899,...,0,0,0,0.0,0,0,0,61,2.0,0.03


In [ ]:
cook1.to_csv("10생산건불량률.csv",index=False,encoding='euc-kr')

## 10개이상 생산건에 관한 불량률

In [ ]:
cook1 = pd.read_csv("10생산건불량률.csv",encoding='euc-kr')

In [ ]:
csort = cook1.drop_duplicates(subset=['품목명'])
csort2 = csort.sort_values('품목별불량률',ascending=False)

In [ ]:
csort2.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,발생시간,오류시간Hour,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명,품목명합계,품목명불량갯수,품목별불량률
976,3,D004,PET병,13A00422,멸치볶음소스300g,W005,69.257405,99.457405,134.250005,23.195,...,8:57,8,10:30,93.0,W005432,제품 넘침 청소,충전실,13,3.0,0.23
465,5,C003,쿠커 #3,23D01143,요거트드레싱(재),W003,71.459688,85.759688,121.448488,23.793,...,0,0,0,0.0,0,0,0,14,3.0,0.21
1143,5,D001,3K 캔,13P03010,시아스피자소스,W005,71.619478,114.119478,149.856378,23.825,...,0,0,0,0.0,0,0,0,14,3.0,0.21
57,22,D004,PET병,13R02052,이마트베스트스테이크소스,W005,71.748776,119.648776,155.279326,23.754,...,0,0,0,0.0,0,0,0,21,4.0,0.19
15450,5,D088,냉동밥 내포장#1,1BA03368,치즈스테이크볶음밥,W005,70.864390,122.964390,158.686190,23.815,...,0,0,0,0.0,0,0,0,28,5.0,0.18
